In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Utilizando o Gemini para recomendação de produtos baseado em avaliação de imagens

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retail/multimodal_retail_recommendations.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Executar no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retail/multimodal_retail_recommendations.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Ver no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retail/multimodal_retail_recommendations.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abrir no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão geral

Para empresas de varejo, os sistemas de recomendação melhoram a experiência do cliente e, portanto, podem aumentar as vendas.

Este notebook mostra como você pode usar os recursos multimodais do modelo Gemini Pro Vision para criar rapidamente um sistema de recomendação multimodal pronto para uso.

## Cenário

O cliente mostra sua sala de estar:

|Foto do cliente |
|:-----:|
|<img src="https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/rooms/spacejoy-c0JoR_-2x3E-unsplash.jpg" width="80%">  |



Abaixo há quatro opções de cadeiras que o cliente está tentando decidir em sua compra:

|Cadeira 1| Cadeira 2 | Cadeira 3 | cadeira 4 |
|:-----:|:----:|:-----:|:----:|
| <img src="https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/cesar-couto-OB2F6CsMva8-unsplash.jpg" width="80%">|<img src="https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/daniil-silantev-1P6AnKDw6S8-unsplash.jpg" width="80%">|<img src="https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/ruslan-bardash-4kTbAMRAHtQ-unsplash.jpg" width="80%">|<img src="https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/scopic-ltd-NLlWwR4d3qU-unsplash.jpg" width="80%">|


Você terá aqui um exercício de como utilizar o `Gemini Pro Vision` para acelerar a solução deste tipo de cenário.

### Objetivos

Seu principal objetivo é aprender como criar um sistema de recomendação que possa fornecer recomendações e explicações usando um modelo multimodal: `Gemini Pro Vision`.

Neste notebook, você começará com uma cena (por exemplo, uma sala de estar) e usará o modelo `Gemini Pro Vision` para realizar a compreensão visual. Você também investigará como o modelo `Gemini Pro Vision` pode ser usado para recomendar um item (por exemplo, uma cadeira) de uma lista de itens de mobiliário como entrada.

Neste notebook você aprenderá:
- como usar o modelo `Gemini Pro Vision` para realizar a compreensão visual
- como levar em consideração a multimodalidade ao solicitar o modelo `Gemini Pro Vision`
- como o modelo `Gemini Pro Vision` pode ser usado para criar aplicativos de recomendação de varejo prontos para uso

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.


## Primeiros passos

### Instale a SDK da Vertex AI

In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if 'google.colab' in sys.modules:

#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if 'google.colab' in sys.modules:

#     # Define project information
#     PROJECT_ID = "[your-project-id]" # @param {type:"string"}
#     LOCATION = "us-central1" # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importando as bibliotecas necessárias

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.generative_models import GenerativeModel, Image

## Importando o modelo `Gemini 1.0 Pro Vision`

O modelo Gemini Pro Vision `gemini-1.0-pro-vision` é um modelo multimodal que suporta a adição de imagem e vídeo em texto ou prompts de chat para uma resposta de texto.

In [ ]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### Defina algumas funções auxiliares

Defina funções auxiliares para carregar e exibir imagens.

In [ ]:
import http.client
import io
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

def display_image(image: Image,
                  max_width: int = 600,
                  max_height: int = 350) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # Modes such as RGBA are not yet supported by all Jupyter environments
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    display_image_compressed(pil_image)


def display_image_compressed(pil_image: PIL_Image.Image) -> None:
    image_io = io.BytesIO()
    pil_image.save(image_io, "jpeg", quality=80, optimize=True)
    image_bytes = image_io.getvalue()
    ipython_image = IPython.display.Image(image_bytes)
    IPython.display.display(ipython_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        if response.headers["Content-Type"] not in ("image/png", "image/jpeg"):
            raise Exception("Image can only be in PNG or JPEG format")
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_image(content)
        else:
            print(content)

### Compreensão visual com `Gemini 1.0 Pro Vision`

Aqui você pedirá ao modelo Gemini Pro Vision para descrever uma sala em detalhes a partir de sua imagem. Para fazer isso você precisa **combinar texto e imagem em um único prompt**.

In [ ]:
# urls for room images
room_image_url = "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/rooms/spacejoy-c0JoR_-2x3E-unsplash.jpg"

# load room images as Image Objects
room_image = load_image_from_url(room_image_url)

prompt = "Descreva o que é visível nesta sala e sua atmosfera geral:"
contents = [
    prompt,
    room_image,
]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

### Gerando recomendações abertas com base em conhecimento integrado

Usando a mesma imagem, você pode pedir ao modelo que recomende **um móvel** que caiba nele junto com a descrição do ambiente.

Observe que o modelo pode escolher **qualquer móvel** recomendado neste caso, e pode fazê-lo a partir de seu único conhecimento integrado.

In [ ]:
prompt1 = "Recomende um novo móvel para esta sala:"
prompt2 = "e explique a razão em detalhes"
contents = [
    prompt1,
    room_image,
    prompt2
]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

Na próxima célula, você pedirá ao modelo que recomende **um tipo de cadeira** que caiba nela junto com a descrição do ambiente.

Observe que o modelo pode escolher **qualquer tipo de cadeira** para recomendar neste caso.

In [ ]:
prompt1 = "Descreva esta sala:"
prompt2 = "e recomende um tipo de cadeira que combine com ela"
contents = [
    prompt1,
    room_image,
    prompt2
]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

### Gerando recomendações com base nas imagens fornecidas

Em vez de manter a recomendação aberta, você também pode fornecer uma lista de itens para o modelo escolher. 

Aqui você baixará algumas imagens de cadeiras e as definirá como opções para o modelo Gemini recomendar. Isto é particularmente útil para empresas de varejo que desejam fornecer recomendações aos usuários com base no tipo de quarto que possuem e nos itens disponíveis que a loja oferece.

In [ ]:
# Download and display sample chairs
furniture_image_urls = [
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/cesar-couto-OB2F6CsMva8-unsplash.jpg",
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/daniil-silantev-1P6AnKDw6S8-unsplash.jpg",
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/ruslan-bardash-4kTbAMRAHtQ-unsplash.jpg",
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/scopic-ltd-NLlWwR4d3qU-unsplash.jpg",
]

# Load furniture images as Image Objects
furniture_images = [load_image_from_url(url) for url in furniture_image_urls]

# To recommend an item from a selection, you will need to label the item number within the prompt.
# That way you are providing the model with a way to reference each image as you pose a question.
# Labelling images within your prompt also help to reduce hallucinations and overall produce better results.
contents = [
    "Considere as seguintes cadeiras:",
    "cadeira 1:", furniture_images[0],
    "cadeira 2:", furniture_images[1],
    "cadeira 3:", furniture_images[2],
    "cadeira 4:", furniture_images[3], 
    "sala:",
    room_image,
    "Você é um designer de interiores. Para cada cadeira, explique se ela seria apropriada ou não para o estilo da sala. Sumarize quais as duas melhores opções de cadeira:",
]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

Você pode também pedir a resposta em algum formato específico, como JSON por exemplo, para simplificar a integração da resposta do modelo com algum sistema de recomendações já existente em seu ambiente:

In [ ]:
contents = [
    "Considere as seguintes cadeiras:",
    "cadeira 1:", furniture_images[0],
    "cadeira 2:", furniture_images[1],
    "cadeira 3:", furniture_images[2],
    "cadeira 4:", furniture_images[3], 
    "sala:",
    room_image,
    "Você é um designer de interiores. Retorne em JSON, para cada cadeira, se ela seria uma boa escolha ou não para esta sala, incluindo uma explicação da decisão:",
]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Resposta--------")
for response in responses:
    print(response.text, end="")

## Conclusão

Este notebook mostrou como você pode construir facilmente um sistema de recomendação multimodal usando Gemini para móveis, mas também pode usar uma abordagem semelhante em:

- recomendar roupas com base em uma ocasião ou imagem do local
- recomendação de papel de parede com base na sala e nas configurações

Você também pode querer explorar como você pode construir um sistema RAG, onde você recupera imagens relevantes do inventário de sua loja para usuários que podem usar o Gemini para ajudar a identificar a escolha mais ideal entre as várias opções fornecidas, e também explicar a lógica aos usuários.